# Pescador demo

This notebook illustrates some of the basic functionality of [pescador](https://github.com/bmcfee/pescador): a package to facilitate iterative learning from data streams (implemented as python generators).

In [1]:
import pescador

import numpy as np
np.set_printoptions(precision=4)
import sklearn
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics

In [2]:
def data_generator(X, Y, m=20, scale = 1e-1):
    '''A gaussian noise generator for data
    
    Parameters
    ----------
    X : ndarray
        features, n_samples by dimensions
        
    Y : ndarray
        labels, n_samples
        
    m : int
        size of the minibatches to generate
        
    scale : float > 0
        scale of the noise to add
        
    Generates
    ---------
    batch
        An infinite stream of batch dictionaries
        batch = dict(X=X[i], Y=Y[i])
    '''
    
    X = np.atleast_2d(X)
    Y = np.atleast_1d(Y)

    
    n, d = X.shape
    
    while True:
        i = np.random.randint(0, n, size=m)
        
        noise = scale * np.random.randn(m, d)
        
        yield {'X': X[i] + noise, 'Y': Y[i]}

In [3]:
# Load up the iris dataset for the demo
data = sklearn.datasets.load_iris()
X, Y = data.data, data.target
classes = np.unique(Y)

In [4]:
# What does the data stream look like?

# First, we'll wrap the generator function in a Streamer object.
# This is necessary for a few reasons, notably so that we can re-instantiate
# the generator multiple times (eg once per epoch)

stream = pescador.Streamer(data_generator, X, Y)

# The BufferedStreamer class takes any Streamer as input, and
# carves it into batches of up to buffer_size (3, in this case) samples
# the buffer size can be larger or smaller than the native size of the input batches
for q in pescador.BufferedStreamer(stream, 3).generate(max_batches=3):
    print(q)

{'X': array([[ 5.0841,  3.4094,  1.6031,  0.661 ],
       [ 6.4516,  2.6899,  5.3236,  1.8523],
       [ 5.1093,  3.4024,  1.4215,  0.2673]]), 'Y': array([0, 2, 0])}
{'X': array([[ 6.2578,  3.1387,  5.2456,  2.2445],
       [ 6.3599,  2.7933,  5.4971,  1.9203],
       [ 6.0606,  2.2571,  5.121 ,  1.5348]]), 'Y': array([2, 2, 2])}
{'X': array([[ 5.6751,  2.6475,  3.8321,  1.1055],
       [ 7.6561,  2.8387,  6.5888,  1.8972],
       [ 5.2579,  3.6381,  1.622 ,  0.1337]]), 'Y': array([1, 2, 0])}


# Benchmarking
We can benchmark our learner's efficiency by running a couple of experiments on the Iris dataset.

Our classifier will be L1-regularized logistic regression.

In [5]:
%%time
ss = sklearn.model_selection.ShuffleSplit(n_splits=2, test_size=0.2)
for train, test in ss.split(np.arange(len(X))):
    
    # Make an SGD learner, nothing fancy here
    classifier = sklearn.linear_model.SGDClassifier(verbose=0, 
                                                    loss='log',
                                                    penalty='l1', 
                                                    n_iter=1)
    
    # Again, build a streamer object
    stream = pescador.Streamer(data_generator, X[train], Y[train])
    
    # And train the model on the stream.
    n_steps = 0
    for batch in stream.generate(max_batches=5e3):
        classifier.partial_fit(batch['X'], batch['Y'], classes=classes)
        
        n_steps += 1
    
    # How's it do on the test set?
    print('Test-set accuracy: {:.3f}'.format(sklearn.metrics.accuracy_score(Y[test], classifier.predict(X[test]))))
    print('# Steps: ', n_steps)

Test-set accuracy: 0.967
# Steps:  5000
Test-set accuracy: 1.000
# Steps:  5000
CPU times: user 7.48 s, sys: 43.8 ms, total: 7.53 s
Wall time: 7.73 s


# Parallelism

It's possible that the learner is more or less efficient than the data generator.  If the data generator has higher latency than the learner (SGDClassifier), then this will slow down the learning.

Pescador uses zeromq to parallelize data stream generation, effectively decoupling it from the learner.

In [6]:
%%time
ss = sklearn.model_selection.ShuffleSplit(n_splits=2, test_size=0.2)
for train, test in ss.split(np.arange(len(X))):
    
    # Make an SGD learner, nothing fancy here
    classifier = sklearn.linear_model.SGDClassifier(verbose=0, 
                                                    loss='log',
                                                    penalty='l1', 
                                                    n_iter=1)
    
    # First, turn the data_generator function into a Streamer object
    stream = pescador.Streamer(data_generator, X[train], Y[train])
    
    # Then, send this thread to a second process
    zmq_stream = pescador.ZMQStreamer(stream, 5156)
    
    # Run the output through a second buffer for mini-batch training
#     bufferd_stream = pescador.BufferedStreamer(zmq_stream, 20)
    
    # And train the model on the stream.
    n_steps = 0
    for batch in zmq_stream.generate(max_batches=5e3):
        classifier.partial_fit(batch['X'], batch['Y'], classes=classes)
        
        n_steps += 1
    
    # How's it do on the test set?
    print('Test-set accuracy: {:.3f}'.format(sklearn.metrics.accuracy_score(Y[test], classifier.predict(X[test]))))
    print('# Steps: ', n_steps)

Test-set accuracy: 0.933
# Steps:  5000
Test-set accuracy: 1.000
# Steps:  5000
CPU times: user 7.45 s, sys: 111 ms, total: 7.56 s
Wall time: 7.67 s
